In [8]:
import sys
sys.path.append('../src/')
import time
import os
import copy

import numpy as np
import tensorly as tn
import matlab.engine

from loewnerize_acts import loewnerize_acts
import ll1_tools

## Load raw data for LenetLinear classifier model

In [2]:
targets_path = '../data/LenetLinear/activations/test/targets.npy'
targets = np.load(targets_path)

inputs_path = '../data/LenetLinear/activations/test/99/layers.layer_0.linear.npy'
inputs_raw = np.load(inputs_path)
print('Inputs shape')
print(inputs_raw.shape)

outputs_path = '../data/LenetLinear/activations/test/99/layers.layer_0.rat.npy'
outputs_raw = np.load(outputs_path)
print('Outputs shape')
print(outputs_raw.shape)
print(outputs_raw[0,0,0])

# tensorlab_path = '~/ware/matlab_tools/tensorlab'
tensorlab_path = '~/tools/matlab_tools/Tensorlab'

Inputs shape
(39, 256, 10)
Outputs shape
(39, 256, 10)
-5.9573436


In [3]:
lwn_tensors = loewnerize_acts(inputs_raw, outputs_raw, targets, tensorlab_path)
datapath = '../data/LenetLinear/activations/test/99/loewnerized_acts'
np.savez(datapath, *lwn_tensors)

In [10]:
for lwn_trgt in lwn_tensors:
    print(lwn_trgt.shape)
    print(type(lwn_trgt))

(5, 5, 979)
<class 'numpy.ndarray'>
(5, 5, 1133)
<class 'numpy.ndarray'>
(5, 5, 1030)
<class 'numpy.ndarray'>
(5, 5, 1008)
<class 'numpy.ndarray'>
(5, 5, 980)
<class 'numpy.ndarray'>
(5, 5, 890)
<class 'numpy.ndarray'>
(5, 5, 956)
<class 'numpy.ndarray'>
(5, 5, 1027)
<class 'numpy.ndarray'>
(5, 5, 973)
<class 'numpy.ndarray'>
(5, 5, 1008)
<class 'numpy.ndarray'>


## perform (L,L,1) decomposition - per target (0-9)

In [9]:
eng = matlab.engine.start_matlab()
s = eng.genpath(tensorlab_path)
eng.addpath(s, nargout=0)

num_terms = 400
rat_deg = 3

L = rat_deg * np.ones(num_terms)
Us = list()
outs = list()

tic = time.perf_counter()
for tns in lwn_tensors:
    tmp_U, tmp_out = eng.ll1(tns, L, nargout=2)
    Us.append(tmp_U)
    outs.append(tmp_out)
toc = time.perf_counter()
print(toc-tic)

Index in position 3 exceeds array bounds. Index must not exceed 25.

Error in mlsvd_rsi (line 109)
    S = Y(ind{:});

Error in ll1 (line 356)
    [V,T] = options.Compression(T,size_core,options.CompressionOptions);



MatlabExecutionError: 
  File /Users/matthewmerris/tools/Matlab_tools/Tensorlab/mlsvd_rsi.m, line 109, in mlsvd_rsi

  File /Users/matthewmerris/tools/Matlab_tools/Tensorlab/ll1.m, line 356, in ll1
Index in position 3 exceeds array bounds. Index must not exceed 25.


## perform (L,L,1) decomposition - entire collection

In [ ]:
super_lwn = 